In [31]:
from langchain.indexes import VectorstoreIndexCreator 
from langchain_community.utilities import ApifyWrapper 
from langchain_core.documents import Document
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

In [32]:
llm = ChatOpenAI(model_name = "gpt-4o-mini", temperature = 0)

In [33]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override = True)

True

In [34]:
apify = ApifyWrapper()

In [35]:
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")

In [37]:
loader = apify.call_actor(
    actor_id = "apify/website-content-crawler", 
    run_input = {"startUrls": [{"url": "https://www.weforum.org/agenda/2024/05/ways-ai-can-benefit-education/"}]}, # Example URL used: https://python.langchain.com/v0.2/docs/introduction/
    dataset_mapping_function = lambda item: Document(
        page_content = item["text"] or "", metadata = {"source": item["url"]}
    )
)

In [44]:
# Here, the embedding argument for VectorstoreIndexCreator is needed

index = VectorstoreIndexCreator(embedding = embedding).from_loaders([loader])

print(help(index))
print(index.__str__)

/home/mpariaszevski/anaconda3/envs/AutoGPT_Project/lib/python3.11/site-packages/langchain/indexes/vectorstore.py:127: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  )


Help on VectorStoreIndexWrapper in module langchain.indexes.vectorstore object:

class VectorStoreIndexWrapper(pydantic.v1.main.BaseModel)
 |  VectorStoreIndexWrapper(*, vectorstore: langchain_core.vectorstores.base.VectorStore) -> None
 |  
 |  Wrapper around a vectorstore for easy access.
 |  
 |  Method resolution order:
 |      VectorStoreIndexWrapper
 |      pydantic.v1.main.BaseModel
 |      pydantic.v1.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  async aquery(self, question: str, llm: Optional[langchain_core.language_models.base.BaseLanguageModel] = None, retriever_kwargs: Optional[Dict[str, Any]] = None, **kwargs: Any) -> str
 |      Query the vectorstore.
 |  
 |  async aquery_with_sources(self, question: str, llm: Optional[langchain_core.language_models.base.BaseLanguageModel] = None, retriever_kwargs: Optional[Dict[str, Any]] = None, **kwargs: Any) -> dict
 |      Query the vectorstore and get back sources.
 |  
 |  query(self, questio

In [39]:
query = "What is the affect of AI on education?" 

output = index.query_with_sources(llm = llm, question = query)

In [40]:
print(output)
print("#" * 50)
print(output["answer"]) 
print(output["sources"])

{'question': 'What is the affect of AI on education?', 'answer': 'AI has the potential to significantly transform education by improving student assessment and guidance, fostering digital literacy, critical thinking, creativity, and problem-solving skills. It offers a more personalized approach to teaching and learning, tailored to individual student needs and preferences. AI can also enhance educational outcomes by making learning more inclusive, particularly for students with disabilities, through the creation of adaptable digital materials. Furthermore, AI can help manage continuous assessments, provide targeted feedback, and enable teachers to focus on more strategic aspects of education, leading to improved learning outcomes.\n\n', 'sources': ''}
##################################################
AI has the potential to significantly transform education by improving student assessment and guidance, fostering digital literacy, critical thinking, creativity, and problem-solving skil